In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class customizedModule(nn.Module):
    def __init(self):
        super(customizedModule,self).__init()
    def customizedLinear(self,in_dim,out_dim,activation=None,dropout=False):
        c1 = nn.Sequential(nn.Linear(in_dim,out_dim))
        nn.init.xavier_uniform_(c1[0].weight)
        nn.init.constant_(c1[0].bias,0)
        
        if activation is not None:
            c1.add_module(str(len(c1)),activation)
        if dropout:
            c1.add_module(str(len(c1)),nn.Dropout(p=self.args.dropout))  
        return c1

In [62]:
class CrossAttention(customizedModule):
    def __init__(self,dx,dq,mode):
        super(CrossAttention,self).__init__()
        self.w1 = self.customizedLinear(dx,dx)
        self.w2 = self.customizedLinear(dq,dx)   
        self.w1[0].bias.requires_grad = False
        self.w2[0].bias.requires_grad = False
        
        # bias for add attention
        self.wt = self.customizedLinear(dx,1,activation= nn.Sigmoid())
        self.wt[0].bias.requires_grad = False
        self.bsa = nn.Parameter(torch.zeros(dx))  
        # 'mul' or 'add'
        self.mode = mode      
    def forward(self,x,q):
        if self.mode is 'mul':
            # W(1)x W(2)c
            x = self.w1(x)
            q = self.w2(q)  
            # <x,q>
            s = x*q
            # s = [a0,a1,a2...]
            s = torch.sum(s,dim=1)
            # softmax along row
            s = F.softmax(s,dim=0)
            s = torch.reshape(s,(s.size(0),-1))
            s = s*x
            return s
        elif self.mode is 'add':
            
            wx = self.w1(x)
            wq = self.w2(q)
            print(wx,'wx\n')
            print(wq,'wq\n')
            
            print('---wx+wq+bsa-----------\n')
            print(wx+wq+self.bsa)
            print('----wt--------\n')
            p =  self.wt(wx+wq+self.bsa)
            p = F.softmax(p,dim = 0)
            print(p)
            print('----sm-------\n')
            p = F.softmax(p,dim=0)
            
            return p
        else:
            raise NotImplementedError('CrossAttention error:<mul or add>')

In [63]:

x = torch.randn(10,5)
q = torch.Tensor([0.3,2,0.55,0.44,0.49])
model = CrossAttention(x.size(-1),q.size(-1),'add')
model(x,q)


tensor([[-6.6977e-01, -4.7398e-01,  3.3411e-01, -6.8279e-01,  4.4025e-01],
        [ 3.8512e-01, -7.0845e-01, -1.5288e+00,  1.1687e+00, -1.3726e+00],
        [-2.4917e-02, -1.3510e-01,  2.9440e-04,  3.6954e-01, -9.1932e-01],
        [-1.4013e-01,  8.8569e-01,  6.5329e-01,  2.9766e-01,  5.5337e-01],
        [-3.2299e-01,  8.7515e-01,  2.3928e-01,  5.2562e-01,  4.0810e-01],
        [-1.4465e-02, -6.6849e-01, -1.5744e+00,  1.0605e+00, -1.1455e+00],
        [-3.9434e-01,  3.9155e-01,  2.7453e-02, -3.4362e-02,  9.1491e-01],
        [-7.5008e-01, -8.2309e-01, -4.8446e-01, -1.4956e-01, -7.7293e-01],
        [ 1.8929e+00,  1.0909e+00,  1.0614e+00,  5.4501e-01, -5.2964e-01],
        [-6.2779e-01, -3.0593e-01,  8.3267e-01, -1.0657e+00,  2.0863e-01]],
       grad_fn=<AddmmBackward>) wx

tensor([ 0.4825, -1.4595, -0.7528,  0.0733,  0.2653], grad_fn=<AddBackward0>) wq

---wx+wq+bsa-----------

tensor([[-0.1873, -1.9335, -0.4187, -0.6095,  0.7055],
        [ 0.8676, -2.1680, -2.2816,  1.2420, -1.107

tensor([[0.0974],
        [0.1037],
        [0.1019],
        [0.0983],
        [0.0988],
        [0.1031],
        [0.0973],
        [0.1005],
        [0.1017],
        [0.0974]], grad_fn=<SoftmaxBackward>)

![](https://i.imgur.com/ONXvoZC.png)

In [64]:
class CSA(customizedModule):
    def __init__(self,dx,dq):
        super(CSA,self).__init__()
        self.dx = dx
        self.dq = dq
        
        # Wsa
        self.Wsa1 = self.customizedLinear(dx,dx)
        self.Wsa2 = self.customizedLinear(dq,dx)
        self.Wsa1[0].bias.requires_grad = False
        self.Wsa2[0].bias.requires_grad = False
        
       
        self.crossAttention = CrossAttention(dx,dq,'mul')
    
    def forward(self,x,q):
        # x(seq_len,word_dim) q(word_dim)
        x = self.crossAttention(x,q)
        gi = self.Wsa1(x)
        gj = self.Wsa2(y)
        
        